# Benchmark huggingface and vllm inference on macos

vllm doesn't support MPS on macOS, as off Dec 2025. So its performance is abysmal even compared to huggingface generate.

In [3]:
import time

In [1]:
import torch
from dawnet.inspector import LLMInspector
from dawnet.tokens import Tokens
from dawnet import op

torch.set_grad_enabled(False)

torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [2]:
model_id = "Qwen/Qwen3-4B-Thinking-2507"
device = torch.device("mps")
insp = LLMInspector.from_hf(model_id).to(device=device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
PROMPT = """What is (234 + 413) * 89?"""
output = insp.generate(PROMPT, chat=True, use_original=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
import vllm

In [6]:
from vllm import LLM, SamplingParams

INFO 12-14 03:02:42 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
vmodel = LLM(model=model_id)

INFO 12-14 03:02:45 [utils.py:253] non-default args: {'disable_log_stats': True, 'model': 'Qwen/Qwen3-4B-Thinking-2507'}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO 12-14 03:02:52 [model.py:514] Resolved architecture: Qwen3ForCausalLM
INFO 12-14 03:02:52 [model.py:1636] Using max model len 262144
WARNING 12-14 03:02:52 [cpu.py:164] VLLM_CPU_KVCACHE_SPACE not set. Using 256.00 GiB for KV cache.
INFO 12-14 03:02:52 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=4096.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO 12-14 03:02:55 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.
(EngineCore_DP0 pid=14772) INFO 12-14 03:02:56 [core.py:93] Initializing a V1 LLM engine (v0.13.0rc2.dev104+g4fa7ce46f.d20251213) with config: model='Qwen/Qwen3-4B-Thinking-2507', speculative_config=None, tokenizer='Qwen/Qwen3-4B-Thinking-2507', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=262144, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cpu, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:01<00:02,  1.46s/it]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.16it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.09it/s]
(EngineCore_DP0 pid=14772) 


(EngineCore_DP0 pid=14772) INFO 12-14 03:03:03 [default_loader.py:308] Loading weights took 2.77 seconds
(EngineCore_DP0 pid=14772) INFO 12-14 03:03:03 [kv_cache_utils.py:1291] GPU KV cache size: 1,864,064 tokens
(EngineCore_DP0 pid=14772) INFO 12-14 03:03:03 [kv_cache_utils.py:1296] Maximum concurrency for 262,144 tokens per request: 7.11x
(EngineCore_DP0 pid=14772) INFO 12-14 03:03:26 [cpu_model_runner.py:65] Warming up model for the compilation...
(EngineCore_DP0 pid=14772) INFO 12-14 03:03:55 [cpu_model_runner.py:75] Warming up done.
(EngineCore_DP0 pid=14772) INFO 12-14 03:03:55 [core.py:259] init engine (profile, create kv cache, warmup model) took 51.86 seconds
(EngineCore_DP0 pid=14772) WARNING 12-14 03:03:56 [vllm.py:629] Inductor compilation was disabled by user settings,Optimizations settings that are only active duringInductor compilation will be ignored.
(EngineCore_DP0 pid=14772) WARNING 12-14 03:03:56 [cpu.py:164] VLLM_CPU_KVCACHE_SPACE not set. Using 256.00 GiB for KV c

In [8]:
voutputs = vmodel.generate(PROMPT, sampling_params)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output:

KeyboardInterrupt: 